In [1]:
!pip install crewai
!pip install requests
!pip install langchain_google_genai
!pip install crewai-tools # Install the correct package
#!pip install llamaindex



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 570.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 258.2 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 55.0 MB/s eta 0:00:00

In [2]:
import os
import crewai
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    SerperDevTool,
    WebsiteSearchTool,
    ScrapeWebsiteTool
)
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
#from llamaindex import Document, GPTIndex, SimpleRetriever

# Retrieve your Google API key
google_api_key = userdata.get('Gemini_API')
os.environ["GOOGLE_API_KEY"] = google_api_key
# Retrieve your Serper API key
serper_api_key = userdata.get('SERPER_API_KEY')
os.environ["SERPER_API_KEY"] = serper_api_key

# Initialize Gemini model
gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    verbose=True,
    temperature=0.5,
    google_api_key=google_api_key
)

# Initialize SerperDevTool
#serper = SerperDevTool(api_key=serper_api_key)

# Instantiate tools
search_tool = SerperDevTool(api_key=serper_api_key)
scrape_tool = ScrapeWebsiteTool()

# **Retrieve and set your OpenAI API key**
openai_api_key = userdata.get('OPENAI_API_KEY')  # Replace 'OPENAI_API_KEY' with the actual key name in userdata
os.environ["OPENAI_API_KEY"] = openai_api_key  # Set the OpenAI API key in the environment variables


In [4]:
agent = Agent(
    role='Dermatologist',
    goal='Provide accurate and helpful dermatology-related information and advice.',
    backstory="""You are a dermatologist specializing in providing information and advice on skin conditions,
    treatments, and skincare practices. Your goal is to assist users with their dermatology-related queries
    by leveraging both search results and advanced language models.""",
    verbose=True,
    #verbose=False,
    llm=gemini,
    tools=[search_tool],
    allow_delegation=True,
    cache=False,  # Disable cache for this agent
)

task = Task(
    description=(
        "Handle dermatology-related user queries with RAG and search. "
        "Provide a concise response in 1 paragraph, with bullets for each item. "
        "If the response is a list, include a brief definition for each item. "
        "The output should not contain any website URLs."
    ),
    expected_output=(
        "A concise list without URLs."
    ),
    agent=agent,
    tools=[search_tool],
    llm=gemini,
    #verbose=True,
    verbose=False,
    human_input=True
)

from crewai import Process
crew = Crew(
    agents=[agent],
    tasks=[task],
    #verbose=2,
    verbose=False,
    process=Process.sequential
)


question = ""
inputs = {"question": question}


results = crew.kickoff(inputs=inputs)
# Print the results

#final_result = results.results[-1].output
print("######################")
print(results)








> Entering new CrewAgentExecutor chain...
This is the agent's final answer: Please provide me with your dermatology-related question. I'm ready to help!
Please provide feedback: What causes Eczema?
Thought: I need to search the internet for information on the causes of eczema.

Action: Search the internet
Action Input: {'search_query': 'What causes eczema'} 


Search results: Title: Eczema: What It Is, Symptoms, Causes, Types & Treatment
Link: https://my.clevelandclinic.org/health/diseases/9998-eczema
Snippet: Eczema is a common skin condition that causes a skin rash, dry skin and itchiness. There's no cure, but there are treatments to manage your symptoms.
---
Title: Eczema (Atopic Dermatitis): Causes and Treatment
Link: https://nationaleczema.org/eczema/
Snippet: Eczema is an inflammatory skin condition that causes dry skin, itchy skin, rashes, scaly patches, blisters and skin infections.
---
Title: Atopic dermatitis (eczema) - Symptoms and causes - Mayo Clinic
Link: https://www.ma

In [9]:
results.raw

'Eczema, also known as atopic dermatitis, is a common skin condition that causes inflammation, dryness, itchiness, and rashes. The exact cause is unknown, but it is believed to be a combination of genetic and environmental factors. Here are some potential causes:\n\n* **Genetics:** Eczema often runs in families, suggesting a genetic predisposition.\n* **Defective skin barrier:** People with eczema have a weaker skin barrier, making them more susceptible to irritants and allergens.\n* **Immune system:** The immune system may overreact to certain triggers, leading to inflammation and skin symptoms.\n* **Environmental factors:**  Triggers like allergens (dust mites, pollen, pet dander), irritants (soaps, detergents, harsh chemicals), and stress can worsen eczema symptoms.'